# 1. Libs

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import investpy as inv
from cvxopt import matrix, solvers
import pandas_datareader.data as web
import matplotlib.pyplot as plt
from datetime import date

current_year = date.today().year
ten_year = current_year 

# 2. Functions

In [2]:
def optimize_portfolio(n, avg_ret, covariance, r_min):
    P = covariance
    # x = variable(n)
    q = matrix(np.zeros((n, 1)), tc='d')
    # inequality constraints Gx <= h
    # captures the constraints (avg_ret'x >= r_min) and (x >= 0)
    G = matrix(np.concatenate((
        -np.transpose(np.array(avg_ret)), 
        -np.identity(n)), 0))
    h = matrix(np.concatenate((
        -np.ones((1,1))*r_min, 
        np.zeros((n,1))), 0))
    # equality constraint Ax = b; captures the constraint sum(x) == 1
    A = matrix(1.0, (1,n))
    b = matrix(1.0)
    sol = solvers.qp(P, q, G, h, A, b)
    return sol

# 3. Code

In [3]:
n = 10 #number tickets

In [4]:
# Importing YFinance and overriding pandas_datareader methods
yf.pdr_override()

In [5]:
br = inv.stocks.get_stocks(country='brazil') # brazil as a database
br = br.loc[[i[-2:] != '11' for i in br['symbol']]] # removendo fundos (terminação 11)
tickers = br['symbol'] + ".SA" # adding ".SA" to conform to the base

In [6]:
prices = dict()
dayly_return_mean = dict()

one_month = 21

for ticker in tickers.to_list():
    prices[ticker] = web.get_data_yahoo(ticker, start=f'{ten_year}-01-01').dropna() #getting 10 year data for each paper
    prices[ticker]["Daily Return"] = prices[ticker]["Adj Close"].pct_change(1) #calculate daily return
    dayly_return_mean[ticker] = prices[ticker]["Daily Return"].iloc[-one_month:].mean() #calculate mean daily return
dayly_return_mean = pd.DataFrame([dayly_return_mean]).sort_values(by=0, axis=1, ascending=False) #put in no ascending 

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BBRK3.SA: No

C:\Users\FILIPE~1\AppData\Local\Temp/ipykernel_27256/3587900995.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prices[ticker]["Daily Return"] = prices[ticker]["Adj Close"].pct_change(1) #calculate daily return


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- LAME3.SA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- LAME4.SA: No data found, symbol may be delisted
[*********************100%***********************]  1 of 1 completed
[*********************100%******

In [7]:
ten_tickers = dayly_return_mean.iloc[:,1:(n+1)].columns.to_list() #name of ten tickers
dayly_return_mean = dayly_return_mean.loc[:,ten_tickers] #select only ten tickers in base

In [8]:
# calculating stock returns
returns = pd.DataFrame()
for ticker in ten_tickers:
    returns[ticker] = prices[ticker]["Daily Return"]
returns.dropna(inplace=True)

#### Statistical information

In [9]:
covariance = np.array(returns.cov())

In [10]:
volatility  = returns.iloc[-22:].pct_change(1)/100
volatility = volatility.tail(volatility.shape[0] -1)*(one_month**0.5)

In [11]:
# expected return threshold
r_min = 0.005

#### Quadratic Programming

In [12]:
avg_ret = dayly_return_mean.to_numpy().T
P = matrix(covariance)
q = matrix(np.zeros((n, 1)))
# inequality constraints Gx <= h
# captures the constraints (avg_ret'x >= r_min) and (x >= 0)
G = matrix(np.concatenate((
             -np.transpose(np.array(avg_ret)), 
             -np.identity(n)), 0))
h = matrix(np.concatenate((
             -np.ones((1,1))*r_min, 
              np.zeros((n,1))), 0))

In [13]:
# equality constraint Ax = b; captures the constraint sum(x) == 1
A = matrix(np.ones((1,n)))
b = matrix(1.0)

In [14]:
sol = solvers.qp(P, q, G, h, A, b)

     pcost       dcost       gap    pres   dres
 0:  1.3546e-04 -1.0126e+00  1e+00  3e-16  4e+00
 1:  1.3544e-04 -1.0143e-02  1e-02  7e-17  4e-02
 2:  1.3296e-04 -1.1378e-04  2e-04  6e-17  9e-04
 3:  1.0103e-04  7.5769e-05  3e-05  8e-17  2e-19
 4:  9.2613e-05  8.9830e-05  3e-06  8e-17  8e-20
 5:  9.1160e-05  9.1038e-05  1e-07  8e-17  6e-20
 6:  9.1063e-05  9.1061e-05  1e-09  7e-17  1e-19
Optimal solution found.


#### More informations

In [15]:
# standard deviation of  portfolio
std_portfolio = 100*(sol['primal objective']**0.5)

In [16]:
# calculate % of each ticker in portfolio
delta = pd.DataFrame(np.array(sol['x']).T, columns=[s.rstrip('.SA') for s in ten_tickers])*100

In [17]:
# save portfolio as list
portfolio = np.array(delta).tolist()[0]

In [18]:
# Put delta in string format and make your visualization
delta = delta.round(6).astype(str) + "%"

In [32]:
# Calculate sharp ratio
log_return = np.ma.masked_invalid(np.sum(np.log(returns.iloc[-21:]/returns.iloc[-21:].shift())*portfolio, axis=1))
sharpe_ratio = log_return.mean()/log_return.std()

In [34]:
sharpe_ratio

0.06606702560731958

In [33]:
volatility

,FSLR34.SA,BKBR3.SA,SNSY5.SA,MDIA3.SA,RSID3.SA,PLAS3.SA,PETR3.SA,CIEL3.SA,AZUL4.SA,PETR4.SA
Date,,,,,,,,,,
2022-07-22,-0.089124,-0.017848,-0.045826,-0.031654,-0.030909,-inf,-0.090835,-0.137032,0.037139,-0.141022
2022-07-25,-0.045826,0.069958,inf,-0.068665,0.136057,-0.045826,0.138193,-0.068963,-0.034975,0.154554
2022-07-26,-inf,0.004391,-0.091343,-0.275321,-0.106941,inf,-0.030547,0.045380,0.104668,-0.035919
2022-07-27,-0.178192,-0.125478,-0.161167,0.083272,-0.102320,0.157241,-0.025121,0.135706,-0.111512,0.003932
2022-07-28,0.234528,0.028484,-0.054838,-0.075483,-0.008931,-0.066789,0.103181,-0.051341,-0.030452,0.079495
2022-07-29,0.000746,-0.031096,0.138090,-0.031970,0.054218,-0.028621,0.093164,-0.391127,-0.062437,0.042211
2022-08-01,-0.045174,0.280655,0.047069,-0.141177,-0.034050,-0.045826,-0.054708,-0.069439,-0.111681,-0.056775
2022-08-02,0.407891,-0.036925,-0.098817,0.077858,0.088557,NaN,-0.047843,-0.086661,-0.191228,-0.060655
2022-08-03,-0.109107,0.058970,-0.035952,-0.039036,-0.002899,NaN,-0.503827,0.231218,-0.141497,-0.039743
